Ian J. Goodfellow, Jonathon Shlens & Christian Szegedy
ICLR 2015

- [Intriguing properties of neural networks (ICLR 2014b)](http://arxiv.org/abs/1312.6199) prove:
	-  Box-constrained L-BFGS can reliably find adversarial examples
	- The same adversarial example is often misclassified by a variety of classifiers with different architectures or trained on different subsets of the training data
	- Training on adversarial examples can regularize the model - however, this was not practical at the time due to the need for expensive constrained optimization in the inner loop

- **For linear models**: vector dot product for computing activations, when given sample with additive noise, is given by $$ w^T\tilde{x} = w^Tx + w^T\eta =  w^Tx + \epsilon ||w||_1 $$ when $ \eta = -\epsilon \times sign(w) $. Notice that when average magnitude of weight vector elements is *m* and dimensionality is *n*, this additive activations scales with *n* even though the $ ||\eta||_1 = \epsilon $. We can think of this as a sort of **accidental steganography**, where a linear model is forced to attend exclusively to the signal that aligns most closely with its weights, even if multiple signals are present and other signals have much greater amplitude.

-   LSTMs, ReLUs, and maxout networks are all intentionally designed to behave in very linear ways, so that they are easier to optimize.  More nonlinear models such as sigmoid networks are carefully tuned to spend most of their time in the non-saturating, more linear regime for the same reason. **This linear behavior suggests that cheap, analytical perturbations of a linear model should also damage neural networks**

- **Fast Gradient Sign Method**: For current value of parameters, linearize the cost function, to get max norm constrained perturbation: $$ \mathbf{\eta} = \epsilon sign(\nabla_xJ(\theta, x, y)) $$

- **FGSM for logistic regression**: The log loss for the adversarial training reduces to: $$ \mathbb{E}_{(x,y) \sim data}[\zeta(\ y\ (\ \epsilon ||w||_1 - w^Tx - b\ ))] $$ This lookes like *L1 regularization*. But L1 is  weight decay is more “worst case” than adversarial training, because it fails to deactivate in the case of good margin.

- **Adversarial Training of Deep Networks**: Augment objective function with the adversarial loss term: $$ \tilde{J}(\theta, x, y) = \alpha\ J(\theta, x, y) + (1-\alpha)\ J(\theta, x + \epsilon sign(\nabla_xJ(\theta, x, y)), y) $$ [for exp, they set $ \alpha = 0.5 $].
	- This approach means that we continually update our supply of adversarial examples, to make them resist the current version of the model
	- Noticeable performance boost with and without adv training. Since error rate on adversarial samples was not decreasing, authors increased model capacity. Consistent improvement
	- Adversarial examples are transferable between the two models but with the adversarially trained model showing greater robustness
	- When the adversarially trained model does misclassify an adversarial example, its predictions are unfortunately still highly confident
	- We also found that the weights of the learned model changed significantly, with the weights of the adversarially trained model being significantly more localized and interpretable

- The adversarial training procedure can be seen as **minimizing the worst case error when the data is perturbed by an adversary**.  That can be interpreted as learning to play an adversarial game, or as minimizing an upper bound on the expected cost over noisy samples with noise from$ U(-\epsilon,\epsilon) $ added to the inputs.

- Adversarial training can also be seen as a **form of active learning, where the model is able to request labels on new points**. In this case the human labeler is replaced with a heuristic labeler that copies labels from nearby points.

- Because the derivative of the sign function is zero or undefined everywhere, gradient descent on the adversarial objective function based on the **fast gradient sign method does not allow the model to anticipate how the adversary will react to changes in the parameters**.

- **Perturbations to the hidden layers (intermediate activations)**: Our view of adversarial training is that it is only clearly  useful when the model has the capacity to learn to resist adversarial examples. This is only clearly the case when a universal approximator theorem applies. Because the last layer of a neural network, the linear-sigmoid or linear-softmax layer, is not a universal approximator of functions of the final hidden layer, this suggests that one is likely to encounter problems with underfitting when applying adversarial perturbations to the final hidden layer. We indeed found this effect. **Our best results with training using perturbations of hidden layers never involved perturbations of the final hidden layer**
	- On models with unbounded activations (e.g. ReLU), hidden layer perturbation shows negligle difference since such models simply respond by making their hidden unit activations very large, so it is usually better to just perturb the original input.

- **Why adversarial examples generalize**
	- Unlike non-linear view, linear view means that adversarial samples occurs in broad subspaces of the input space. Thus, there can be broad overlap between the adversarial regions of input space for different models.
	- To explain why mutiple classifiers assignthe same class to adversarial examples, we hypothesize that neural networks trained with current methodologies all resemble the linear classifier learned on the same training set. This hypothesis **does not explain** all of the maxout network’s mistakes or all of the mistakes that generalize across models, but clearly a significant proportion of them are consistent with linear behavior being a major cause of cross-model generalization.

- Alternate hypotheses:
	- *Generative training could provide more constraint on the training process, or cause the model to learn what to distinguish “real” from “fake” data and be confident only on “real” data:* It remains possible that some other form of generative training could confer resistance, but clearly the mere fact of being generative is not alone sufficient
	- *Individual models have strange quirks but averaging over many models can cause adversarial examples to wash out:* Ensembling provides only limited resistance to adversarial perturbation. In particular, samples can be found that perturn the entire ensemble

**This work has shown we can partially correct for this problem by explicitly identifying problematic points and correcting the model at each of these points. However, one may also conclude that the model families we use are intrinsically flawed. Ease of optimization has come at the cost of models that are easily misled. This motivates the development of optimization procedures that are able to train models whose behavior is more locally stable.**